# Import libraries and generate token


In [135]:
import requests
import json
import pandas as pd
from pandas.io.json import json_normalize
import numpy as np
import geopy
from geopy.geocoders import Nominatim
from geopy.point import Point
from cryptography.fernet import Fernet
from datetime import datetime



headers = {
    'accept': 'application/json',
    'Content-Type': 'application/json',
}

json_data = {
    'full_name': 'Fabiano Moreira Alves',
    'email': 'fabianomalves@proton.me',
}

access_token = requests.post('https://begrowth.deta.dev/user/', headers=headers, json=json_data)
access_token_json = access_token.json()

print(access_token_json)


{'user': 'fabianomalves@proton.me', 'API Token': 'BG6B5YOZO857'}


# Split the user and key as dictionary to get the acess token

In [136]:
# split dictionary into keys and values
keys = []
values = []
items = access_token_json.items()
for item in items:
    keys.append(item[0]), values.append(item[1])
 
# printing keys and values separately
string_acess_token = str(values[-1])
print(string_acess_token)


BG6B5YOZO857


# Consume the API, calling the endpoint https://begrowth.deta.dev/token=access_token, concatenatating the url with the token. Then, create a dataframe for the jason and print data frame result.

In [137]:
url_dev = "https://begrowth.deta.dev/token="
url_dev_with_token = url_dev + string_acess_token
print(url_dev_with_token)


https://begrowth.deta.dev/token=BG6B5YOZO857


# Normalize the json data into pandas Data Frame

In [138]:
data = json.loads(requests.get(url_dev_with_token).text)
df_json_normalize = pd.json_normalize(data)
print(df_json_normalize)


        id   first_name   last_name                         email  gender  \
0        1       Kellen     Cowherd            kcowherd0@nasa.gov    Male   
1        2     Josefina     Swalowe        jswalowe1@slashdot.org  Female   
2        3    Priscilla    Prickett           pprickett2@webs.com  Female   
3        3    Priscilla    Prickett           pprickett2@webs.com  Female   
4        4        Leroi    Spinello          lspinello3@google.fr    Male   
...    ...          ...         ...                           ...     ...   
1015   997        Boote      Downse  bdownsero@huffingtonpost.com    Male   
1016   979       Shelly    Storrock        sstorrockr6@census.gov  Female   
1017   998       Cindie  Riddlesden    criddlesdenrp@hubpages.com  Female   
1018   999     Roderick     Alfonso      ralfonsorq@quantcast.com    Male   
1019  1000  Konstantine      Perell         kperellrr@blogger.com    Male   

             utm                                                cpf  \
0   

# Find duplicate rows and filtering them

In [139]:
duplicate_rows = df_json_normalize[df_json_normalize.duplicated(['id', ])]
print(duplicate_rows)


       id first_name    last_name                         email  gender  \
3       3  Priscilla     Prickett           pprickett2@webs.com  Female   
42     25       Flin  Friedenbach       ffriedenbacho@google.pl    Male   
64     62    Ermanno       Golson        egolson1p@so-net.ne.jp    Male   
82     51     Archer       Moreno            amoreno1e@yale.edu    Male   
109    91        Ara     Carrabot     acarrabot2i@wordpress.org  Female   
195   106      Parke       Wooles       pwooles2x@google.com.au    Male   
256   233   Veronika         Skae            vskae6g@meetup.com  Female   
273   266      Weber       Priest            wpriest7d@java.com    Male   
310   273     Trevor     Yosevitz      tyosevitz7k@redcross.org    Male   
360   336    Vanessa       Allsep        vallsep9b@amazon.co.jp  Female   
429   419     Mortie         Lots             mlotsbm@imgur.com    Male   
478   413    Tiphany     Endricci          tendriccibg@cnet.com  Female   
520   497     Ramsey     

# Drop duplicate rows, filtering by 'id'

In [140]:
df_distinct_id_rows = df_json_normalize.drop_duplicates(subset=['id'])
print(df_distinct_id_rows)


        id   first_name    last_name                         email  gender  \
0        1       Kellen      Cowherd            kcowherd0@nasa.gov    Male   
1        2     Josefina      Swalowe        jswalowe1@slashdot.org  Female   
2        3    Priscilla     Prickett           pprickett2@webs.com  Female   
4        4        Leroi     Spinello          lspinello3@google.fr    Male   
5        5     Bellanca      Laycock       blaycock4@amazonaws.com  Female   
...    ...          ...          ...                           ...     ...   
1014   996        Bride  Grzegorczyk     bgrzegorczykrn@flavors.me  Female   
1015   997        Boote       Downse  bdownsero@huffingtonpost.com    Male   
1017   998       Cindie   Riddlesden    criddlesdenrp@hubpages.com  Female   
1018   999     Roderick      Alfonso      ralfonsorq@quantcast.com    Male   
1019  1000  Konstantine       Perell         kperellrr@blogger.com    Male   

             utm                                               

# Doing the reverse geocoding and creating address column for the singles rows

In [145]:
# Create a geocoder object using the Nominatim API
geolocator = Nominatim(user_agent="my_geocoder_state")

# Define a function to reverse geocode the state
def get_state(lat, lng):
  # Use the geocoder object to reverse geocode the coordinates
  location = geolocator.reverse((lat, lng))
  # Extract the state from the response
  state = location.raw['address']['state']
  return state

# Apply the function to each row of the DataFrame and store the result in a new column

df_distinct_id_rows.loc[:, ['address_state']] = df_distinct_id_rows.apply(lambda x: get_state(x['address.geo_latitude'], x['address.geo_longitude']), axis=1)



# Print the resulting DataFrame
df_distinct_id_rows.head()



/tmp/ipykernel_10426/2579216447.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_distinct_id_rows.loc[:, ['address_state']] = df_distinct_id_rows.apply(lambda x: get_state(x['address.geo_latitude'], x['address.geo_longitude']), axis=1)


,id,first_name,last_name,email,gender,utm,cpf,address.geo_latitude,address.geo_longitude,address.country,address_state
0,1,Kellen,Cowherd,kcowherd0@nasa.gov,Male,br-gg-cc,gAAAAABjieY90eD1zlPMbL92LLt-f6j5-apLacUZRGPncA...,-28.258844,-53.500999,Brazil,Rio Grande do Sul
1,2,Josefina,Swalowe,jswalowe1@slashdot.org,Female,br-fb-emp,gAAAAABjieY91IeyDJoBhywKgFQYgUypzOeQeu0Jt9-Yji...,-14.232982,-39.857912,Brazil,Bahia
2,3,Priscilla,Prickett,pprickett2@webs.com,Female,br-fb-emp,gAAAAABjieY9-JZwc8sPwkdEb8clzdybJhbaudiEtZ0CZ9...,-6.683513,-35.148879,Brazil,Paraíba
4,4,Leroi,Spinello,lspinello3@google.fr,Male,br-gg-fin,gAAAAABjieY9nowJ0vS-c0ecOFTv0I2X2QnpE4uPcRDEg5...,-5.017543,-42.511724,Brazil,Piauí
5,5,Bellanca,Laycock,blaycock4@amazonaws.com,Female,br-wpp-fin,gAAAAABjieY9ScaiUDpfi3rPkc52PGwUHB8aUJvDYCQBfd...,-21.248487,-45.001375,Brazil,Minas Gerais


# Saving dataframe into csv file

In [146]:
df_distinct_id_rows.to_csv('../data_engineer_test/df_distinct_id_rows.csv')

# Read csv file

In [147]:
df_distinct_id_rows = pd.read_csv('../data_engineer_test/df_distinct_id_rows.csv')
df_distinct_id_rows.head()

,Unnamed: 0,id,first_name,last_name,email,gender,utm,cpf,address.geo_latitude,address.geo_longitude,address.country,address_state
0,0,1,Kellen,Cowherd,kcowherd0@nasa.gov,Male,br-gg-cc,gAAAAABjieY90eD1zlPMbL92LLt-f6j5-apLacUZRGPncA...,-28.258844,-53.500999,Brazil,Rio Grande do Sul
1,1,2,Josefina,Swalowe,jswalowe1@slashdot.org,Female,br-fb-emp,gAAAAABjieY91IeyDJoBhywKgFQYgUypzOeQeu0Jt9-Yji...,-14.232982,-39.857912,Brazil,Bahia
2,2,3,Priscilla,Prickett,pprickett2@webs.com,Female,br-fb-emp,gAAAAABjieY9-JZwc8sPwkdEb8clzdybJhbaudiEtZ0CZ9...,-6.683513,-35.148879,Brazil,Paraíba
3,4,4,Leroi,Spinello,lspinello3@google.fr,Male,br-gg-fin,gAAAAABjieY9nowJ0vS-c0ecOFTv0I2X2QnpE4uPcRDEg5...,-5.017543,-42.511724,Brazil,Piauí
4,5,5,Bellanca,Laycock,blaycock4@amazonaws.com,Female,br-wpp-fin,gAAAAABjieY9ScaiUDpfi3rPkc52PGwUHB8aUJvDYCQBfd...,-21.248487,-45.001375,Brazil,Minas Gerais


# Decript the CPF column

In [148]:
fernet = Fernet(b'ekkxXo0uHWRkIbHqHrLS4gaMj2hWTYMJyPTAbi9INGI=')

df_distinct_id_rows['cpf'] = df_distinct_id_rows['cpf'].apply(lambda x: fernet.decrypt(x.encode()))





In [151]:
df_distinct_id_rows.head()

,Unnamed: 0,id,first_name,last_name,email,gender,utm,cpf,address.geo_latitude,address.geo_longitude,address.country,address_state
0,0,1,Kellen,Cowherd,kcowherd0@nasa.gov,Male,br-gg-cc,b'91362597210',-28.258844,-53.500999,Brazil,Rio Grande do Sul
1,1,2,Josefina,Swalowe,jswalowe1@slashdot.org,Female,br-fb-emp,b'72137209969',-14.232982,-39.857912,Brazil,Bahia
2,2,3,Priscilla,Prickett,pprickett2@webs.com,Female,br-fb-emp,b'37166330112',-6.683513,-35.148879,Brazil,Paraíba
3,4,4,Leroi,Spinello,lspinello3@google.fr,Male,br-gg-fin,b'86876744655',-5.017543,-42.511724,Brazil,Piauí
4,5,5,Bellanca,Laycock,blaycock4@amazonaws.com,Female,br-wpp-fin,b'72461816018',-21.248487,-45.001375,Brazil,Minas Gerais


# Save the decrypted data result to a csv file

In [150]:

df_distinct_id_rows.to_csv('../data_engineer_test/df_decrypt_cpf.csv')


# Read the csv decrypted data

In [152]:
df_decrypt_cpf = pd.read_csv('../data_engineer_test/df_decrypt_cpf.csv')
df_decrypt_cpf.head()
#df_decrypt_cpf.info()

,Unnamed: 0.1,Unnamed: 0,id,first_name,last_name,email,gender,utm,cpf,address.geo_latitude,address.geo_longitude,address.country,address_state
0,0,0,1,Kellen,Cowherd,kcowherd0@nasa.gov,Male,br-gg-cc,b'91362597210',-28.258844,-53.500999,Brazil,Rio Grande do Sul
1,1,1,2,Josefina,Swalowe,jswalowe1@slashdot.org,Female,br-fb-emp,b'72137209969',-14.232982,-39.857912,Brazil,Bahia
2,2,2,3,Priscilla,Prickett,pprickett2@webs.com,Female,br-fb-emp,b'37166330112',-6.683513,-35.148879,Brazil,Paraíba
3,3,4,4,Leroi,Spinello,lspinello3@google.fr,Male,br-gg-fin,b'86876744655',-5.017543,-42.511724,Brazil,Piauí
4,4,5,5,Bellanca,Laycock,blaycock4@amazonaws.com,Female,br-wpp-fin,b'72461816018',-21.248487,-45.001375,Brazil,Minas Gerais


# Formating the cpf column

In [153]:
df_decrypt_cpf['cpf'] = df_decrypt_cpf['cpf'].apply(lambda x: x[2: -1])


In [154]:
df_decrypt_cpf.head()

,Unnamed: 0.1,Unnamed: 0,id,first_name,last_name,email,gender,utm,cpf,address.geo_latitude,address.geo_longitude,address.country,address_state
0,0,0,1,Kellen,Cowherd,kcowherd0@nasa.gov,Male,br-gg-cc,91362597210,-28.258844,-53.500999,Brazil,Rio Grande do Sul
1,1,1,2,Josefina,Swalowe,jswalowe1@slashdot.org,Female,br-fb-emp,72137209969,-14.232982,-39.857912,Brazil,Bahia
2,2,2,3,Priscilla,Prickett,pprickett2@webs.com,Female,br-fb-emp,37166330112,-6.683513,-35.148879,Brazil,Paraíba
3,3,4,4,Leroi,Spinello,lspinello3@google.fr,Male,br-gg-fin,86876744655,-5.017543,-42.511724,Brazil,Piauí
4,4,5,5,Bellanca,Laycock,blaycock4@amazonaws.com,Female,br-wpp-fin,72461816018,-21.248487,-45.001375,Brazil,Minas Gerais


# Inserting columns 

In [155]:
now = datetime.now()

# Create new columns
df_decrypt_cpf = df_decrypt_cpf.assign(
    dt_insert=now,
    candidate_name='Fabiano Moreira Alves'
)


In [156]:
df_decrypt_cpf.head()

,Unnamed: 0.1,Unnamed: 0,id,first_name,last_name,email,gender,utm,cpf,address.geo_latitude,address.geo_longitude,address.country,address_state,dt_insert,candidate_name
0,0,0,1,Kellen,Cowherd,kcowherd0@nasa.gov,Male,br-gg-cc,91362597210,-28.258844,-53.500999,Brazil,Rio Grande do Sul,2022-12-26 12:43:04.838078,Fabiano Moreira Alves
1,1,1,2,Josefina,Swalowe,jswalowe1@slashdot.org,Female,br-fb-emp,72137209969,-14.232982,-39.857912,Brazil,Bahia,2022-12-26 12:43:04.838078,Fabiano Moreira Alves
2,2,2,3,Priscilla,Prickett,pprickett2@webs.com,Female,br-fb-emp,37166330112,-6.683513,-35.148879,Brazil,Paraíba,2022-12-26 12:43:04.838078,Fabiano Moreira Alves
3,3,4,4,Leroi,Spinello,lspinello3@google.fr,Male,br-gg-fin,86876744655,-5.017543,-42.511724,Brazil,Piauí,2022-12-26 12:43:04.838078,Fabiano Moreira Alves
4,4,5,5,Bellanca,Laycock,blaycock4@amazonaws.com,Female,br-wpp-fin,72461816018,-21.248487,-45.001375,Brazil,Minas Gerais,2022-12-26 12:43:04.838078,Fabiano Moreira Alves


# Removing Unnamed columns

In [158]:
del df_decrypt_cpf['Unnamed: 0']
del df_decrypt_cpf['Unnamed: 0.1']
df_decrypt_cpf.head()

,id,first_name,last_name,email,gender,utm,cpf,address.geo_latitude,address.geo_longitude,address.country,address_state,dt_insert,candidate_name
0,1,Kellen,Cowherd,kcowherd0@nasa.gov,Male,br-gg-cc,91362597210,-28.258844,-53.500999,Brazil,Rio Grande do Sul,2022-12-26 12:43:04.838078,Fabiano Moreira Alves
1,2,Josefina,Swalowe,jswalowe1@slashdot.org,Female,br-fb-emp,72137209969,-14.232982,-39.857912,Brazil,Bahia,2022-12-26 12:43:04.838078,Fabiano Moreira Alves
2,3,Priscilla,Prickett,pprickett2@webs.com,Female,br-fb-emp,37166330112,-6.683513,-35.148879,Brazil,Paraíba,2022-12-26 12:43:04.838078,Fabiano Moreira Alves
3,4,Leroi,Spinello,lspinello3@google.fr,Male,br-gg-fin,86876744655,-5.017543,-42.511724,Brazil,Piauí,2022-12-26 12:43:04.838078,Fabiano Moreira Alves
4,5,Bellanca,Laycock,blaycock4@amazonaws.com,Female,br-wpp-fin,72461816018,-21.248487,-45.001375,Brazil,Minas Gerais,2022-12-26 12:43:04.838078,Fabiano Moreira Alves


# Alter data columns to '_' instead '.'.

In [159]:
df_decrypt_cpf_ready = df_decrypt_cpf.rename(columns=lambda x: x.replace('.', '_'))

df_decrypt_cpf_ready.head()


,id,first_name,last_name,email,gender,utm,cpf,address_geo_latitude,address_geo_longitude,address_country,address_state,dt_insert,candidate_name
0,1,Kellen,Cowherd,kcowherd0@nasa.gov,Male,br-gg-cc,91362597210,-28.258844,-53.500999,Brazil,Rio Grande do Sul,2022-12-26 12:43:04.838078,Fabiano Moreira Alves
1,2,Josefina,Swalowe,jswalowe1@slashdot.org,Female,br-fb-emp,72137209969,-14.232982,-39.857912,Brazil,Bahia,2022-12-26 12:43:04.838078,Fabiano Moreira Alves
2,3,Priscilla,Prickett,pprickett2@webs.com,Female,br-fb-emp,37166330112,-6.683513,-35.148879,Brazil,Paraíba,2022-12-26 12:43:04.838078,Fabiano Moreira Alves
3,4,Leroi,Spinello,lspinello3@google.fr,Male,br-gg-fin,86876744655,-5.017543,-42.511724,Brazil,Piauí,2022-12-26 12:43:04.838078,Fabiano Moreira Alves
4,5,Bellanca,Laycock,blaycock4@amazonaws.com,Female,br-wpp-fin,72461816018,-21.248487,-45.001375,Brazil,Minas Gerais,2022-12-26 12:43:04.838078,Fabiano Moreira Alves


# Saving the final transformed dataframe into df_decrypt_cpf_ready.csv

In [ ]:
df_decrypt_cpf_ready.to_csv('../data_engineer_test/df_decrypt_cpf_ready.csv')